In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

#### tf-idf with spars matrix

In [2]:
cols = np.array(['shop_name','target'])
vals = np.array([['la bella mela',10],['la brutta bestia',12],['de goude tip',5],['de goude lul',19]])
train_df = pd.DataFrame(data=vals,columns=cols)

In [3]:
print(train_df)

          shop_name target
0     la bella mela     10
1  la brutta bestia     12
2      de goude tip      5
3      de goude lul     19


In [4]:
train_df.iloc[:,0]

0       la bella mela
1    la brutta bestia
2        de goude tip
3        de goude lul
Name: shop_name, dtype: object

In [5]:
tf_idf_feats={}
vectorizer = TfidfVectorizer()
tf_idf_feats[train_df.columns[0]] = vectorizer.fit_transform(train_df.iloc[:,0])

In [6]:
vectorizer.vocabulary_

{'bella': 0,
 'bestia': 1,
 'brutta': 2,
 'de': 3,
 'goude': 4,
 'la': 5,
 'lul': 6,
 'mela': 7,
 'tip': 8}

In [7]:
tf_idf_df = pd.DataFrame(data=tf_idf_feats['shop_name'].todense(),columns=vectorizer.get_feature_names())

In [8]:
tf_idf_df

,bella,bestia,brutta,de,goude,la,lul,mela,tip
0,0.617614,0.000000,0.000000,0.000000,0.000000,0.486934,0.000000,0.617614,0.000000
1,0.000000,0.617614,0.617614,0.000000,0.000000,0.486934,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.526405,0.526405,0.000000,0.000000,0.000000,0.667679
3,0.000000,0.000000,0.000000,0.526405,0.526405,0.000000,0.667679,0.000000,0.000000


In [9]:
print("%s\n%s" %(train_df.shape,tf_idf_feats['shop_name'].shape))

(4, 2)
(4, 9)


In [17]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [18]:
from scipy.sparse import hstack
tf_idf_features = [tf_idf_feats[name] for name in sorted(tf_idf_feats)]
sparse_tf_idf_features = hstack(tf_idf_features, format='csr').astype(np.float32)
train_df= downcast_dtypes(train_df)
all_features_sparse = hstack([train_df] + [sparse_tf_idf_features])
print(all_features_sparse.shape)

TypeError: no supported conversion for types: (dtype('O'), dtype('float32'))

In [12]:
from scipy.sparse import hstack
train_contact_df = hstack([train_df.values,tf_idf_feats['shop_name']],format='csr').astype(np.float32)
print(train_contact_df.shape)

TypeError: no supported conversion for types: (dtype('O'), dtype('float64'))